# 3D Stage design
A turbine or compressor stage consists of stator-rotor pairs or rotor-rotor pairs. You can chain together multiple stages to design the entire Turbine using this code. Also control the space between the blade rows. 

For the purpose of this tutorial. 
Technical Definitions used: 
- Airfoils are 2D
- Blades are 3D and made up of airfoils.
- Stage is made up of Blades
- Full Turbine Design is made up of multiple stages 

This tutorial takes you through a Full turbine design consisting of multiple stages

> __Note__: The 1D design of a Turbine or compressor is absolutely critical and must not be overlooked. 3D design optimization without changing the inlet and exit flow angles (Velocity triangles) can result in 1-2% efficiency improvement. **Greater improvements can be achieved by optimizing the 1D.**

In [ ]:
# Installation
!pip install pyturbo-aero

## Creating a Stator Blade
Stator are non-rotating. The purpose of the stator is to guide the flow into the rotor in such a way that maximizes the power and efficiency. The design is govern by the velocity triangles found using 1D design or streamline design approach. 

Below is an example of creating a stator blade 



## Creating a Rotor Blade 